## Vanishing/Exploding gradients problems


In [1]:
import keras


keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

<Dense name=dense, built=False>

In [2]:
he_avg_init = keras.initializers.VarianceScaling(
    scale=2, mode="fan_avg", distribution="uniform"
)
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)


<Dense name=dense_1, built=False>

In [3]:
leaky_relu = keras.layers.LeakyReLU(alpha=0.2)
layer = keras.layers.Dense(10, activation=leaky_relu, kernel_initializer="he_normal")


d:\software\anaconda\envs\AI-related\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [4]:
layer = keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")

## batch normalization


In [5]:
model = keras.models.Sequential(
    [
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(10, activation="softmax"),
    ]
)

d:\software\anaconda\envs\AI-related\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 784)            │         3,136 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 300)            │         1,200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 271,346 (1.04 MB)

 Trainable params: 268,978 (1.03 MB)

 Non-trainable params: 2,368 (9.25 KB)

In [7]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('gamma', True),
 ('beta', True),
 ('moving_mean', False),
 ('moving_variance', False)]

In [8]:
model = keras.models.Sequential(
    [
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("elu"),
        keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
        keras.layers.Activation("elu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(10, activation="softmax"),
    ]
)

In [9]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)

## transfer learning


In [10]:
import numpy as np
import keras
import tensorflow as tf

In [11]:
(X_train_full, y_train_full), (X_test, y_test) = (
    keras.datasets.fashion_mnist.load_data()
)
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [12]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6)  # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2  # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(
        np.float32
    )  # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B))


(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [13]:
print(X_train_A.shape)
print(X_train_B.shape)

(43986, 28, 28)
(200, 28, 28)


In [14]:
tf.random.set_seed(42)
np.random.seed(42)

In [15]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [16]:
model_A.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=["accuracy"],
)

In [17]:
history = model_A.fit(
    X_train_A, y_train_A, epochs=20, validation_data=(X_valid_A, y_valid_A)
)

Epoch 1/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8167 - loss: 0.5598 - val_accuracy: 0.8739 - val_loss: 0.3749
Epoch 2/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8828 - loss: 0.3438 - val_accuracy: 0.8852 - val_loss: 0.3265
Epoch 3/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8931 - loss: 0.3107 - val_accuracy: 0.8931 - val_loss: 0.3032
Epoch 4/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8994 - loss: 0.2926 - val_accuracy: 0.8971 - val_loss: 0.2896
Epoch 5/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9032 - loss: 0.2802 - val_accuracy: 0.8994 - val_loss: 0.2804
Epoch 6/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9068 - loss: 0.2709 - val_accuracy: 0.9023 - val_loss: 0.2733
Epoch 7/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9099 - loss: 0.2634 - val_accuracy: 0.9058 - val_loss: 0.2679
Epoch 8/20
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9114 - loss: 0.2573 - 

In [18]:
model_A.save("my_model_A.keras")

In [19]:
model_A = keras.models.load_model("my_model_A.keras")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [20]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [21]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [22]:
history = model_B_on_A.fit(
    X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B)
)

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=1e-4)
model_B_on_A.compile(
    loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"]
)
history = model_B_on_A.fit(
    X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B)
)

Epoch 1/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5050 - loss: 1.0644 - val_accuracy: 0.6663 - val_loss: 0.7899
Epoch 2/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7500 - loss: 0.6270 - val_accuracy: 0.7789 - val_loss: 0.4981
Epoch 3/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8150 - loss: 0.3955 - val_accuracy: 0.8540 - val_loss: 0.3357
Epoch 4/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8850 - loss: 0.2679 - val_accuracy: 0.8996 - val_loss: 0.2425
Epoch 1/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9200 - loss: 0.2126 - val_accuracy: 0.9047 - val_loss: 0.2322
Epoch 2/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9350 - loss: 0.2031 - val_accuracy: 0.9189 - val_loss: 0.2229
Epoch 3/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9450 - loss: 0.1946 - val_accuracy: 0.9290 - val_loss: 0.2146
Epoch 4/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9450 - loss: 0.1869 - val_accuracy: 0.9341 - val_loss: 0.2071
Epoc

In [23]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9755 - loss: 0.1435


[0.14353129267692566, 0.9754999876022339]

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

In [24]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

In [25]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

In [27]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [28]:
optimizer = keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)

d:\software\anaconda\envs\AI-related\lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [29]:
def exponential_decay_fn(epoch):
    return 0.01 * 0.1 ** (epoch / 20)

In [30]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 ** (epoch / s)

    return exponential_decay_fn

In [32]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [33]:
n_epochs = 25
lr_schedule = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=n_epochs,
    validation_data=(X_valid_scaled, y_valid),
    callbacks=[lr_schedule],
)

Epoch 1/25


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/compile_loss/mse/sub/BroadcastGradientArgs defined at (most recent call last):
  File "d:\software\anaconda\envs\AI-related\lib\runpy.py", line 196, in _run_module_as_main

  File "d:\software\anaconda\envs\AI-related\lib\runpy.py", line 86, in _run_code

  File "d:\software\anaconda\envs\AI-related\lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "d:\software\anaconda\envs\AI-related\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "d:\software\anaconda\envs\AI-related\lib\site-packages\ipykernel\kernelapp.py", line 758, in start

  File "d:\software\anaconda\envs\AI-related\lib\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "d:\software\anaconda\envs\AI-related\lib\asyncio\base_events.py", line 603, in run_forever

  File "d:\software\anaconda\envs\AI-related\lib\asyncio\base_events.py", line 1909, in _run_once

  File "d:

In [34]:
def exponential_decay_fn(epoch, learning_rate):
    return learning_rate * 0.1 ** (1 / 20)

In [35]:
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

In [36]:
lr_schedule = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

In [37]:
s = 20 * len(X_train)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.01)
optimizer = keras.optimizers.SGD(learning_rate)

## avoiding overfitting


In [ ]:
layer = keras.layers.Dense(
    100,
    activation="elu",
    kernel_initializer="he_normal",
    kernel_regularizer=keras.regularizers.l2(0.01),
)

In [38]:
from functools import partial


RegularizedDense = partial(
    keras.layers.Dense,
    activation="elu",
    kernel_initializer="he_normal",
    kernel_regularizer=keras.regularizers.l2(0.01),
)
model = keras.models.Sequential(
    [
        keras.layers.Flatten(input_shape=[28, 28]),
        RegularizedDense(300),
        RegularizedDense(100),
        RegularizedDense(10, activation="softmax", kernel_initializer="glorot_uniform"),
    ]
)

d:\software\anaconda\envs\AI-related\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [39]:
model = keras.models.Sequential(
    [
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(10, activation="softmax"),
    ]
)

d:\software\anaconda\envs\AI-related\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [41]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
y_probas = np.stack([model(X_test_scaled, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_std = y_probas.std(axis=0)

In [ ]:
np.round(model.predict(X_test_scaled[:1]), 2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


array([[0.17, 0.  , 0.22, 0.01, 0.05, 0.06, 0.38, 0.03, 0.01, 0.07]],
      dtype=float32)

In [ ]:
np.round(y_probas[:, :1], 2)

array([[[0.03, 0.  , 0.2 , 0.  , 0.07, 0.01, 0.66, 0.02, 0.  , 0.  ]],

       [[0.23, 0.  , 0.02, 0.02, 0.  , 0.02, 0.04, 0.06, 0.  , 0.6 ]],

       [[0.82, 0.  , 0.02, 0.03, 0.  , 0.04, 0.01, 0.03, 0.02, 0.02]],

       [[0.14, 0.  , 0.26, 0.01, 0.14, 0.04, 0.25, 0.01, 0.  , 0.15]],

       [[0.5 , 0.  , 0.05, 0.01, 0.01, 0.01, 0.4 , 0.01, 0.  , 0.01]],

       [[0.35, 0.  , 0.15, 0.01, 0.02, 0.18, 0.2 , 0.01, 0.01, 0.06]],

       [[0.17, 0.  , 0.37, 0.  , 0.06, 0.05, 0.33, 0.01, 0.01, 0.01]],

       [[0.27, 0.  , 0.07, 0.  , 0.01, 0.  , 0.65, 0.  , 0.  , 0.  ]],

       [[0.52, 0.01, 0.11, 0.  , 0.07, 0.  , 0.11, 0.08, 0.05, 0.04]],

       [[0.03, 0.  , 0.69, 0.03, 0.02, 0.05, 0.08, 0.01, 0.01, 0.08]],

       [[0.01, 0.  , 0.3 , 0.01, 0.05, 0.02, 0.59, 0.  , 0.  , 0.01]],

       [[0.29, 0.  , 0.01, 0.  , 0.01, 0.01, 0.63, 0.04, 0.  , 0.  ]],

       [[0.09, 0.  , 0.05, 0.  , 0.11, 0.07, 0.64, 0.  , 0.  , 0.03]],

       [[0.43, 0.04, 0.07, 0.  , 0.05, 0.16, 0.18, 0.02, 0.02, 0

In [ ]:
np.round(y_proba[:1], 2)

array([[0.22, 0.  , 0.19, 0.01, 0.06, 0.07, 0.3 , 0.05, 0.01, 0.08]],
      dtype=float32)

In [ ]:
y_std = y_probas.std(axis=0)
np.round(y_std[:1], 2)

array([[0.2 , 0.01, 0.2 , 0.02, 0.07, 0.1 , 0.23, 0.09, 0.03, 0.12]],
      dtype=float32)

In [ ]:
y_pred = np.argmax(y_proba, axis=1)
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

np.float64(0.1611)

In [52]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [55]:
keras.layers.Dense(
    100,
    activation="elu",
    kernel_initializer="he_normal",
    kernel_constraint=keras.constraints.max_norm(1.0),
)

<Dense name=dense_23, built=False>